In [11]:
!conda install -c conda-forge hdbscan


Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 22.9.0
  latest version: 23.1.0

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /Users/zacpangan/opt/anaconda3

  added / updated specs:
    - hdbscan


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    conda-22.11.1              |   py39h6e9494a_1         874 KB  conda-forge
    hdbscan-0.8.29             |   py39h7cc1f47_1         567 KB  conda-forge
    python_abi-3.9             |           2_cp39           4 KB  conda-forge
    ruamel.yaml-0.17.21        |   py39ha30fb19_2         173 KB  conda-forge
    ruamel.yaml.clib-0.2.7     |   py39ha30fb19_1         118 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         1.7 MB

The foll

In [3]:
%pip install --upgrade pip setuptools


Defaulting to user installation because normal site-packages is not writeable
  Using cached setuptools-66.1.1-py3-none-any.whl (1.3 MB)
Note: you may need to restart the kernel to use updated packages.


In [1]:
%pip install hdbscan

Defaulting to user installation because normal site-packages is not writeable
  Using cached hdbscan-0.8.29.tar.gz (5.2 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached Cython-0.29.33-py2.py3-none-any.whl (987 kB)
  error: subprocess-exited-with-error
  
  × Building wheel for hdbscan (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [47 lines of output]
      running bdist_wheel
      running build
      running build_py
      creating build
      creating build/lib.macosx-10.9-universal2-cpython-39
      creating build/lib.macosx-10.9-universal2-cpython-39/hdbscan
      copying hdbscan/validity.py -> build/lib.macosx-10.9-universal2-cpython-39/hdbscan
      copying hdbscan/flat.py -> build/lib.macosx-10.9-universal2-cpython-39/hdbscan
      copying hdbscan/__init__.py -> build/lib.macosx-10.9-universal2-cpython-39/hdbscan
      copying hdbscan/prediction.py -> buil

In [1]:
%pip install bertopic
%pip install bertopic[visualization]

Defaulting to user installation because normal site-packages is not writeable
  Using cached bertopic-0.13.0-py2.py3-none-any.whl (103 kB)
  Using cached sentence_transformers-2.2.2-py3-none-any.whl
  Using cached plotly-5.13.0-py2.py3-none-any.whl (15.2 MB)
  Using cached PyYAML-5.4.1-cp39-cp39-macosx_10_9_universal2.whl
  Using cached umap_learn-0.5.3-py3-none-any.whl
  Using cached hdbscan-0.8.29.tar.gz (5.2 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached Cython-0.29.33-py2.py3-none-any.whl (987 kB)
  Using cached tenacity-8.1.0-py3-none-any.whl (23 kB)
  Using cached numba-0.56.4-cp39-cp39-macosx_11_0_arm64.whl (2.4 MB)
  Using cached pynndescent-0.5.8-py3-none-any.whl
  Using cached llvmlite-0.39.1-cp39-cp39-macosx_11_0_arm64.whl (23.1 MB)
  error: subprocess-exited-with-error
  
  × Building wheel for hdbscan (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [

In [ ]:
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('sentiwordnet')
stopwords = stopwords.words('english')

In [9]:
from bertopic import BERTopic
topic_model = BERTopic(min_topic_size=70, n_gram_range=(1,3), verbose=True)

ModuleNotFoundError: No module named 'bertopic'

In [ ]:
def clean_tweets(tweets):
    """
    Remove blank texts, emojis, handles and hashtags, 
    leading and trailing whitespaces, and escape characters.
    """
    tweets['cleaned_text'] = tweets['message'].fillna('') # Remove blank texts
    #tweets['cleaned_text'] = tweets['cleaned_text'].apply(lambda x: emoji.get_emoji_regexp().sub("", x)) # Remove emojis
    tweets['cleaned_text'] = tweets['cleaned_text'].str.replace(r'(@[A-Za-z0-9_]+)|(#[A-Za-z0-9_]+)|(\w+:\/\/\S+)', '', regex=True) # Remove handles and hashtags
    tweets['cleaned_text'] = tweets['cleaned_text'].str.replace(r'\t+|\n+|\r+', '', regex=True)  # Remove escape characters
    tweets['cleaned_text'] = tweets['cleaned_text'].str.replace(r'^\s+|\s+$', '', regex=True)  # Remove trailing and leading whitespaces
    return tweets

# Used for word-level analyses
def superclean_tweets(tweets):
    """
    Lowercase characters, remove non-alphanumeric characters, remove stopwords. 
    """
    tweets['supercleaned_text'] = tweets['cleaned_text'].str.lower() # Transform into lowercase
    tweets['supercleaned_text'] = tweets['supercleaned_text'].str.replace(r'([^A-Za-z0-9_ \t])|(\w+:\/\/\S+)', '') # Remove non-alphanumeric characters
    tweets['supercleaned_text'] = tweets['supercleaned_text'].apply(lambda x: ' '.join([w for w in x.split() if w not in (stopwords)])) # Remove stopwords
    return tweets

In [ ]:
correspondenceTimeline_cleaned = clean_tweets(correspondenceTimeline_preprocessed)
correspondenceTimeline_cleaned = superclean_tweets(correspondenceTimeline_preprocessed)
correspondenceTimeline_cleaned.head()

In [ ]:
from bertopic import BERTopic

# Create a BERTopic model
model = BERTopic(time_slice='1D')

# Fit the model to the data
model.fit(correspondenceTimeline_cleaned)

# Create a DTM for each time slice
#dtm = model.transform(data)

# Access the topics and timestamps for each time slice
topics = model.topic_word_
timestamps = model.timestamps_